<a href="https://colab.research.google.com/github/elenaermakova/NashWardropEquilibrium/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Поиск равновесного распределения транспортных потоков методом нащупывания равновесия Нэша-Вардропа.


**Основные термины:**

- Транспортная сеть представляется графом $\langle V, E \rangle$, где $V$ - множество вершин, $E$ - множество ребер. 
- Множество пар вершин исток-сток $OD \subseteq V \otimes V$.
- Корреспонденция $d_w$ отвечает паре вершин $w \in OD$.
- Множество путей $P_w$ обознает всевозможные пути для пары вершин $w$.
- Поток $x_p$ на пути $p \in P_w$. Множество всех путей $P = \bigcup P_w$.
- Поток $f_e$ по ребру $e$.
- $f_e(x) = \sum_{p \in P} \delta_{e p} x_p$, где $\delta_{e p}$ — идентификатор $e \in p$.
- Затраты на прохождение ребра $e$ описываются функцией $0 \le \tau_e (f_e) \le M$ (ограниченная и неубывающая).
- Затраты на проезд по пути $p$: $G_p(x) = \sum_{e \in E} \tau_e(f_e(x)) \delta_{ep}$
- $X = \{x \ge 0: \sum_{p \in P_w} x_p = d_w, w \in OD\}$
- $\Psi(x) = \sum_{e \in E} \int_{0}^{f_e(x)} \tau_e(z) dz$
- $\nabla \Psi = G$

## Logit-динамика, порождающая стохастическое равновесие

Пользователь $l$ из корреспонденции $w$ на шаге $k + 1$ выбирает путь, который он собирается использовать. С вероятностью $1 - \frac{\lambda}{N}$ путь не меняется, а с вероятностью $\frac{\lambda}{N}$ он выбирает путь относительно зашумленных затрат на предыдущем шаге $p^{l, k + 1} = \arg \max_{q \in P_w} (G_q(x^{k}) + \xi_q^{l, k + 1})$. Независимые случайные величины $\xi_q^{l, k + 1}$ распределены по Гумбелю $P(\xi_q^{l, k + 1} < y) = e^{-e^{-\frac{y}{\gamma} - E}}$. Тогда, если пользователь решает поменять путь, то новый выбирается в соответствии с распределением:
$$ P(p^{l, k + 1} = p) = \frac{\exp \Big(-\frac{1}{\gamma} G_p(x^k)\Big)}{\sum_{q \in P_w} \exp \Big(-\frac{1}{\gamma} G_q(x^k)\Big)}$$



$$ \Psi_{\gamma} (x) = \Psi(x) + \gamma \sum_{w \in W} \sum_{p \in P_w} x_p \ln x_p \rightarrow \min_{x \in X}$$

Так как $\Psi_{\gamma}$ — строго выпуклая, то то стохастическое равновесие всегда единственно.

Итеративный способ нахождения равновесия:

$$ f \rightarrow_{(*)} t \rightarrow_{(**)} x \rightarrow f = \Theta x \rightarrow \dots$$

$$ t_e = \tau_e (f_e) = \overline{t}_e \cdot \Big(1 + \rho \Big(\frac{f_e}{\overline{f}_e}\Big)^{\frac{1}{\mu}}\Big) \quad (*)$$

С адаптивным штрафом:

$$ t_e = \tau_e (f_e) + f_e \tau_e' (f_e) = \overline{t}_e \cdot \Big(1 + \Big(\rho + \frac{1}{\mu}\Big) \Big(\frac{f_e}{\overline{f}_e}\Big)^{\frac{1}{\mu}}\Big)$$

$$ x_p = d_w \cdot \frac{\exp (-\frac{1}{\gamma}\sum_{e \in E} \delta_{ep} t_e)}{\sum_{p' \in P_w} \exp (- \frac{1}{\gamma} \sum_{e \in E} \delta_{ep'} t_e)} \quad (**)$$

## Модель Бэкмана

<figure>
<center>
<img src='https://drive.google.com/uc?id=1Vsl52Fzlzt3vB5amOlLvewAkjgO60qsQ' />
<figcaption>Graph #1</figcaption></center>
</figure>




In [0]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [0]:
delta = np.array([[1, 1, 0, 0, 0], 
                  [1, 0, 1, 0, 1],
                  [0, 0, 0, 1, 1]])
num_p = delta.shape[0]
num_e = delta.shape[1]

In [0]:
def make_plot(x, X, T, xaxes_title='Итерация', line_width=2, mode='lines+markers'):
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Потоки на путях', 
                                                        ('Затраты на путях')))
    fig.add_trace(go.Scatter(x=x, y=X[0], mode=mode, name='path 1', 
                             line=dict(width=line_width + 2), opacity=0.9), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=X[1], mode=mode, name='path 2',
                             line=dict(width=line_width), opacity=0.9), row=1, col=1)
    fig.add_trace(go.Scatter(x=x, y=X[2], mode=mode, name='path 3', 
                             line=dict(width=line_width), opacity=0.9), row=1, col=1)

    fig.add_trace(go.Scatter(x=x, y=T[0], mode=mode, name='path 1', 
                             line=dict(width=line_width + 1), opacity=0.9), row=1, col=2)
    fig.add_trace(go.Scatter(x=x, y=T[1], mode=mode, name='path 2', 
                             line=dict(width=line_width), opacity=0.9), row=1, col=2)
    fig.add_trace(go.Scatter(x=x, y=T[2], mode=mode, name='path 3', 
                             line=dict(width=line_width), opacity=0.9), row=1, col=2)
    fig.update_xaxes(title_text=xaxes_title, row=1, col=1)
    fig.update_xaxes(title_text=xaxes_title, row=1, col=2)
    fig.update_yaxes(title_text="Поток", row=1, col=1)
    fig.update_yaxes(title_text="Затраты", row=1, col=2)
    fig.show()

def experiment(eps=0.01, gamma=1, d=100, mu=1/4, ro=2, ts=[1, 1, 0, 1, 1], 
               fs=[100, 100, 100, 100, 100], f_init=np.array([70, 30, 40, 30, 70]), 
               max_iter=1000, info=True, extra_info=False, figure=False, 
               do_return=False):
    
    def path_flow(t, p):
        logits = [np.exp(-(1/gamma)*np.dot(d, t)) for d in delta]
        x = (d*logits[p])/np.sum(logits)
        return x
    
    def time(fe, e):
        return ts[e]*(1+ro*(fe/fs[e])**(1/mu))
    
    f = f_init

    previous_f = np.array([0]*num_e)
    num_iter = 0
    
    X = []
    TT = []

    while (np.linalg.norm(f - previous_f)) > eps and num_iter <= max_iter:
        previous_f = f
        num_iter += 1
        t = list(map(time, f, range(num_e)))
        x = [path_flow(t, p) for p in range(num_p)]
        X.append(x)
        TT.append([np.dot(delta[p], t) for p in range(num_p)])

        f = np.dot(delta.T, x)

    T = [np.dot(delta[p], t) for p in range(num_p)]
        
    if num_iter < max_iter:
        if info:
            print('Процесс сошелся за {} итераций'.format(num_iter))
            for p in range(num_p):
                print('В равновесии затраты на пути {} равны {}, поток равен {}'.format(p + 1, T[p], x[p]))
            if extra_info:
                for e in range(num_e):
                    print('В равновесии затраты на ребре {} равны {}, поток равен {}'.format(e + 1, t[e], f[e]))

    
    N = np.arange(num_iter)
    X = np.array(X).T
    TT = np.array(TT).T
    
    if figure:
        make_plot(N, X, TT)
    
    if do_return:
      return x, T

Оптимальность каждого из путей в рассматриваемом графе зависит от значений функции $\tau_e$, параметрами которой являются $\overline{t_e}$, $\overline{f_e}$, $\rho$ и $\mu$. Проведем несколько экспериментов со сменой параметра $\overline{t_e}$, оставив при этом остальные параметры неизменными и равными для всех путей. 


*   $\overline{f_e}=(100, 100, 100, 100, 100)^T$
*   $\rho=2$
*   $\mu=1/4$

Различные варианты параметра $\overline{t_e}$:


*   Оптимальными по затратам являются все пути:
    $\overline{t_e}=(1, 2, 1, 2, 1)^T$
*   Пути 1 и 3 являются оптимальными: $\overline{t_e}=(1, 2, 3, 2, 1)^T$
*   Путь два является оптимальным: $\overline{t_e}=(1, 3, 1, 3, 1)^T$
*   Путь 1 является оптимальным: $\overline{t_e}=(0, 1, 2, 1, 2)^T$







In [0]:
experiment(extra_info=True, figure=True, ts=[1, 2, 1, 2, 1])

Процесс сошелся за 8 итераций
В равновесии затраты на пути 1 равны 3.3997282219791174, поток равен 36.22239948460038
В равновесии затраты на пути 2 равны 3.67321461681604, поток равен 27.55520103079924
В равновесии затраты на пути 3 равны 3.3997282219791174, поток равен 36.22239948460038
В равновесии затраты на ребре 1 равны 1.330846765754715, поток равен 63.77760051539962
В равновесии затраты на ребре 2 равны 2.0688814562244024, поток равен 36.22239948460038
В равновесии затраты на ребре 3 равны 1.0115210853066101, поток равен 27.55520103079924
В равновесии затраты на ребре 4 равны 2.0688814562244024, поток равен 36.22239948460038
В равновесии затраты на ребре 5 равны 1.330846765754715, поток равен 63.77760051539962


In [0]:
experiment(extra_info=True, figure=True, ts=[1, 2, 3, 2, 1])

Процесс сошелся за 5 итераций
В равновесии затраты на пути 1 равны 3.351833671576599, поток равен 46.74122911653113
В равновесии затраты на пути 2 равны 5.321963021995005, поток равен 6.5175417669377245
В равновесии затраты на пути 3 равны 3.351833671576599, поток равен 46.74122911653113
В равновесии затраты на ребре 1 равны 1.1609273069676582, поток равен 53.258770883468856
В равновесии затраты на ребре 2 равны 2.1909063646089404, поток равен 46.74122911653113
В равновесии затраты на ребре 3 равны 3.0001084080596883, поток равен 6.5175417669377245
В равновесии затраты на ребре 4 равны 2.1909063646089404, поток равен 46.74122911653113
В равновесии затраты на ребре 5 равны 1.1609273069676582, поток равен 53.258770883468856


In [0]:
experiment(extra_info=True, figure=True, ts=[1, 3, 1, 3, 1])

Процесс сошелся за 14 итераций
В равновесии затраты на пути 1 равны 4.569223682937096, поток равен 28.213454686383916
В равновесии затраты на пути 2 равны 4.134582891268087, поток равен 43.57309062723217
В равновесии затраты на пути 3 равны 4.569223682937096, поток равен 28.213454686383916
В равновесии затраты на ребре 1 равны 1.5312235411589472, поток равен 71.78654531361609
В равновесии затраты на ребре 2 равны 3.038000141778149, поток равен 28.213454686383916
В равновесии затраты на ребре 3 равны 1.0721358089501931, поток равен 43.57309062723217
В равновесии затраты на ребре 4 равны 3.038000141778149, поток равен 28.213454686383916
В равновесии затраты на ребре 5 равны 1.5312235411589472, поток равен 71.78654531361609


In [0]:
experiment(extra_info=True, figure=True, ts=[0, 1, 2, 1, 2])

Процесс сошелся за 26 итераций
В равновесии затраты на пути 1 равны 1.6226278071484739, поток равен 74.69174799175498
В равновесии затраты на пути 2 равны 4.016484310540532, поток равен 6.817638248410567
В равновесии затраты на пути 3 равны 3.0187341855357586, поток равен 18.490613759834446
В равновесии затраты на ребре 1 равны 0.0, поток равен 81.50938624016555
В равновесии затраты на ребре 2 равны 1.6226278071484739, поток равен 74.69174799175498
В равновесии затраты на ребре 3 равны 2.0000863535176445, поток равен 6.817638248410567
В равновесии затраты на ребре 4 равны 1.0023362285128707, поток равен 18.490613759834446
В равновесии затраты на ребре 5 равны 2.0163979570228876, поток равен 25.308252008245013


#Исследование расходимости

До сих пор все параметры, задействованные в итерационной схеме были достаточно удачными, так как во всех случаях метод сходился. Приведем пример, когда метод расходится (например, $\overline{t_e}=(1, 2, 3, 4, 5)^T$).



In [0]:
experiment(extra_info=True, figure=True, ts=[1, 2, 3, 4, 5])

Но уже при $\overline{t_e}=(1, 1, 3, 4, 5)^T$ метод сходится.

In [0]:
experiment(extra_info=True, figure=True, ts=[1, 1, 3, 4, 5])

Процесс сошелся за 11 итераций
В равновесии затраты на пути 1 равны 5.511756541348776, поток равен 96.54958392313631
В равновесии затраты на пути 2 равны 10.774056502638825, поток равен 0.5004521636741421
В равновесии затраты на пути 3 равны 9.000020303309736, поток равен 2.949963913189554
В равновесии затраты на ребре 1 равны 2.774042276385673, поток равен 97.05003608681045
В равновесии затраты на ребре 2 равны 2.7377142649631026, поток равен 96.54958392313631
В равновесии затраты на ребре 3 равны 3.0000000037705545, поток равен 0.5004521636741421
В равновесии затраты на ребре 4 равны 4.00000608082714, поток равен 2.949963913189554
В равновесии затраты на ребре 5 равны 5.000014222482597, поток равен 3.4504160768636964


Этому можно дать следующее объяснение. Выпишем явно, какие приблизительные значения принимает $x$ в расходящемся случае.

*   $x^1=(100, 0, 0)$
*   $x^2=(48, 7, 45)$

Потоки через ребра в случае $x^1$ равны $f=(100, 100, 0, 0, 0)$, в свою очередь затраты на ребрах следующие:

*   $t_1 = 1\times(1+2\times(100/100)^{4})=2$
*   $t_2 = 2\times(1+2\times(100/100)^{4})=6$ $(*)$
*   $t_3 = \overline{t_3}=3$
*   $t_2 = \overline{t_4}=4$
*   $t_2 = \overline{t_5}=5$

Затраты на путях при этом $T=(8, 10, 9)$, то есть они становятся приблизительно равными. Далее заметим, что каждый поток $x_p$ на новой итерации пропорционален $\exp(-{1/\gamma}*T_p)$, что соответствует $x^2$. 
Аналогичные рассчеты для $x^2$:

*   $t_1 = 1\times(1+2\times(55/100)^{4})$
*   $t_2 = 2\times(1+2\times(48/100)^{4})$
*   $t_3 = 3\times(1+2\times(7/100)^{4})$
*   $t_2 = 4\times(1+2\times(45/100)^{4})$
*   $t_2 = 5\times(1+2\times(52/100)^{4})$

В этом случае $t_e\approx\overline{t_e}$, а ввиду указанной выше пропорциональности весь поток перейдет на путь, который по $\overline{t_e}$ был самым оптимальным, что соответствует $x_1$.

Такое поведение в данном случае вызвано тем, что произошло в уравнении $(*)$. Действительно, $t_2$ приняло достаточно большое значение из-за соотношения коэффициетов $\overline{t_2}$ и $\overline{f_2}$, которое позволило сделать затраты на пути 1 приблизительно такими же, как на путях 2 и 3.

Обобщая данный пример, можно сделать следующий вывод: при определенном соотношении параметров, которые входят в вычисление $\exp(-{1/\gamma}*T_p)$, а именно $\overline{t}$, $\overline{f}$, $\rho$, $1/\mu$, $\gamma$, преимущество одного из путей в плане начальных малых затрат может быть скомпенсировано нарастающим потоком, который делает затраты сравнимыми с затратами на других путях. Далее потоки на путях перераспределяются, и путь, потерявший преимущество из-за большого потока, вновь его приобретает.
Покажем, как с помощью изменений $\overline{f}$, $\rho$, $1/\mu$ и $\gamma$ можно устранить расходимость, расмотренную в данном примере.

Увеличение параметра $\gamma$ ведет к более равному перераспределению потоков. по путям на каждой итерации, что не дает достигнуть какому-то пути критического преимущества.

$\gamma: 1\rightarrow 5$

In [0]:
experiment(figure=True, ts=[1, 2, 3, 4, 5], gamma=5)

Процесс сошелся за 10 итераций
В равновесии затраты на пути 1 равны 4.252107084452197, поток равен 59.50769372232077
В равновесии затраты на пути 2 равны 10.026420378970148, поток равен 18.75092250637555
В равновесии затраты на пути 3 равны 9.286547483694108, поток равен 21.741383771303674


Увеличение параметра $\overline{f}$ или уменьшение параметра $\rho$ не позволяет затратам на пути, который изначально имеет преимущество, стать сравнимыми с затратами на остальных путях.

$\overline{f}: (100, 100, 100, 100, 100)\rightarrow (100, 1000, 100, 100, 100)$


$\rho: 2\rightarrow 1$


In [0]:
experiment(figure=True, ts=[1, 2, 3, 4, 5], fs=[100, 1000, 100, 100, 100])

Процесс сошелся за 5 итераций
В равновесии затраты на пути 1 равны 4.876052314869964, поток равен 98.16816667465584
В равновесии затраты на пути 2 равны 10.875682010634922, поток равен 0.24342468132678588
В равновесии затраты на пути 3 равны 9.000001647168704, поток равен 1.5884086440173752


In [0]:
experiment(figure=True, ts=[1, 2, 3, 4, 5], ro=1)

Процесс сошелся за 10 итераций
В равновесии затраты на пути 1 равны 5.558661415428808, поток равен 95.66974402198203
В равновесии затраты на пути 2 равны 9.883066504878057, поток равен 1.2668037610330796
В равновесии затраты на пути 3 равны 9.0000210539423, поток равен 3.063452216984894


Увеличение параметра $\mu$ поспособствует тому, что в итерации, когда нет явного преимущества одного из путей, добавка к $\overline{t}$ не будет пренебрежимо малой.

$\mu: {1/4}\rightarrow 8$

In [0]:
experiment(figure=True, ts=[1, 2, 3, 4, 5], mu=8)

Процесс сошелся за 45 итераций
В равновесии затраты на пути 1 равны 8.999373175355096, поток равен 99.91310029350254
В равновесии затраты на пути 2 равны 17.309952188491195, поток равен 0.024568793407719784
В равновесии затраты на пути 3 равны 16.37897176546788, поток равен 0.06233091308975793


#Эксперименты с параметрами

В предыдущем пункте мы уже исследовали значение параметров задачи в случае, которые могут порождать расходимость метода. Теперь распишем числитель из формулы вычисления потоков немного подробнее $$x_p\sim\exp(-\frac{\rho}{\gamma}\sum_{e\in E}\delta_{ep}\overline{t_e}\Big(\frac{f_e}{\overline{f_e}}\Big)^{1/\mu}-\frac{1}{\gamma}\sum_{e\in E}\delta_{ep}\overline{t_e})$$
Назовем слагаемое $\frac{1}{\gamma}\sum_{e\in E}\delta_{ep}\overline{t_e}$ константной частью, а $\frac{\rho}{\gamma}\sum_{e\in E}\delta_{ep}\overline{t_e}\Big(\frac{f_e}{\overline{f_e}}\Big)^{1/\mu}$ — неконстантной.
Видно, что в зависимости от параметров выбор водителя может очень сильно зависеть от распределения на ребрах $f_e$, а может и не зависеть вовсе в том смысле, что его выбор практически полностью будет определяться константной частью.

Как видно из формулы, увеличение параметра $\gamma$ приводит к тому, что выбор водителя становится все более приближенным к равномерному.

In [0]:
gamma_values = list(range(1, 101))
X, T = [], []

for gamma in gamma_values:
  x, t = experiment(gamma=gamma, info=False, do_return=True, ts=[1, 2, 1, 3, 1])
  X.append(x)
  T.append(t)

X = np.array(X).T
T = np.array(T).T

In [0]:
make_plot(gamma_values, X, T, xaxes_title='$\gamma$', mode='lines')

In [0]:
ro_values = np.arange(1, 300)*0.01
X, T = [], []

for ro in ro_values:
  x, t = experiment(ro=ro, info=False, do_return=True, ts=[1, 2, 1, 3, 1])
  X.append(x)
  T.append(t)

X = np.array(X).T
T = np.array(T).T

In [0]:
make_plot(ro_values, X, T, xaxes_title='$\rho$', mode='lines')

Здесь распределение выбора водителя, как и ожидалось, с увеличением параметра $\rho$ начинало все более определяться константной частью. В определенной точке произошел скачок графика, который указывает на расходимость метода при заданных параметрах.

Уменьшение параметра $\overline(t_e)$ приводит к тому, что водитель с большей вероятностью выберет путь, содержащий данное ребро. Уменьшение параметра $f_e$ приведет к большему влиянию неконстантной части на распределение.

In [0]:
te_values = np.arange(10, 30)
X, T = [], []

for te in te_values:
  x, t = experiment(info=False, do_return=True, ts=[20, te, 10, 20, 20])
  X.append(x)
  T.append(t)

X = np.array(X).T
T = np.array(T).T


In [41]:
make_plot(te_values, X, T, '$\overline{t_e}$', mode='lines')

In [0]:
fe_values = np.arange(80, 100)
X, T = [], []

for fe in fe_values:
  x, t = experiment(info=False, do_return=True, fs=[100, fe, 100, 100, 100])
  X.append(x)
  T.append(t)

X = np.array(X).T
T = np.array(T).T

In [57]:
make_plot(fe_values, X, T, '$\overline{f_e}$', mode='lines')

Теперь перейдем к параметру $\mu$.Уменьшение параметра $\mu$ должно привести к тому, что выбор водителя станет больше определяться константной частью.



In [0]:
mu_values = np.arange(1, 100)*0.01
X, T = [], []

for mu in mu_values:
  x, t = experiment(mu=mu, info=False, do_return=True, ts=[1, 1, 0, 1, 1])
  X.append(x)
  T.append(t)

X = np.array(X).T
T = np.array(T).T

In [0]:
make_plot(mu_values, X, T, '$\mu$', mode='lines')

In [0]:
mu_values = np.arange(1, 100)*0.001
X, T = [], []

for mu in mu_values:
  x, t = experiment(mu=mu, info=False, do_return=True, ts=[1, 1, 0, 1, 1])
  X.append(x)
  T.append(t)

X = np.array(X).T
T = np.array(T).T

In [0]:
make_plot(mu_values, X, T, '$\mu$', mode='lines')

#Изучение влияния адаптивного/неадаптивного штрафа

После упрощения формулы для адаптивного штафа можно прийти к следующему выражению $$ t_e = \tau_e (f_e) + f_e \tau_e' (f_e) = \overline{t}_e \cdot \Big(1 + \Big(\rho + \frac{1}{\mu}\Big) \Big(\frac{f_e}{\overline{f}_e}\Big)^{\frac{1}{\mu}}\Big)$$ 

Отсюда видно, что введение адаптивного штрафа увеличивает параметр $\rho$ на $1/\mu$. Влияние увеличения параметра $\rho$ на потоки на путях уже рассматривалось в предыдущих пунктах.

При неадаптивном штрафе функция затрат на ребрах выглядит следующим образом 
$$t_e = \tau_e (f_e) + С = (\overline{t}_e + С) \Big(1 + \frac{\rho}{(\overline{t}_e + C)} \Big(\frac{f_e}{\overline{f}_e}\Big)^{\frac{1}{\mu}}\Big) = \overline{t}_e \Big(1 + \frac{C}{\overline{t}_e} + \rho \Big(\frac{f_e}{\overline{f}_e}\Big)^{\frac{1}{\mu}}\Big)$$

В этом случае введение неадаптивного штрафа можно рассматривать как изменение следующих параметров:

$\overline{t}_e \rightarrow \overline{t}_e + C$

$\rho \rightarrow \dfrac{\rho}{\overline{t}_e + C}$

Видно, что при этом значение $\rho$ уменьшается, а значение $\overline{t}_e$ увеличивается.
В приведенных выше рассуждениях было выяснено, что уменьшение $\rho$ ведет к уменьшению влияния потоков на ребрах на выбор водителя, а увеличение $\overline{t}_e$ — к увеличению этого влияния в неконстантной части. Таким образом, при достаточно значимом значении потока влияние неадаптивного штрафа на поведение водителей неоднозначно. 
Посмотрим на влияние неадаптивного штрафа при малых значениях потока. В этом случае распределение водителей на путях по большей части зависит от константной части, то есть от параметров $\overline{t_e}$ и $\gamma$. В случае неадаптивного штрафа, как было отмечено, $\overline{t}_e$ переходит в $\overline{t}_e + C$, то есть неадаптивный штраф при малых потоках позволяет перераспределить потоки на путях. 

#Перенос на модель стабильной динамики

К модели стабильной динамики можно перейти от модели Бэкмана с помощью предельного перехода $\mu\rightarrow 0$. В этом случае потоки на путях будут пропорциональны константной части распределения, когда $f_e\lt \overline{f_e}$ (что можно заметить по графику, когда $\mu$ уменьшается). Когда $f_e \gt \overline{f_e}$, неконстантная часть распределения становится бесконечно большой, что приводит к бесконечно малой вероятности выбора водителем пути, который содержит даннное ребро. Если $f_e = \overline{f_e}$, то $x_p\sim exp(-\frac{1}{\gamma}\sum_{e\in E}\delta_{ep}(1+\rho)\overline{t_e})$, то есть выбор водителя зависит только от параметров и не зависит от потоков.

